In [1]:
from pinecone import Pinecone, ServerlessSpec
import os
import pandas as pd
import dotenv



c:\Users\LAKSHAN\AppData\Local\Programs\Python\Python310\lib\site-packages\pinecone\data\index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [2]:
dotenv.load_dotenv()

True

In [3]:
Pinecone_api_key=os.getenv("PINECONE_API")
PINECONE_INDEX=os.getenv("coffee_shop")

In [4]:
Pinecone_api_key

'pcsk_BAJZr_Cdb2QGVpBpVhYzEHXbYW76wnP8AWYLYG3xcxmpunN55hqs2bTcs2DazrfCZBsSr'

In [5]:
pc = Pinecone(api_key=Pinecone_api_key)

In [7]:
from langchain_community.embeddings import HuggingFaceBgeEmbeddings

model_name = "BAAI/bge-small-en"
model_kwargs = {"device": "cpu"}
encode_kwargs = {"normalize_embeddings": True}
hf = HuggingFaceBgeEmbeddings(
    model_name=model_name, model_kwargs=model_kwargs, encode_kwargs=encode_kwargs
)


In [8]:
def get_embedding(model_name,model_kwargs,encode_kwargs,prompt):
    hf = HuggingFaceBgeEmbeddings(
    model_name=model_name, model_kwargs=model_kwargs, encode_kwargs=encode_kwargs
)
    embed=hf.embed_query(prompt)

    return embed

# Wrangle Datasets


In [9]:
df=pd.read_json('../products/products.jsonl',lines=True)

In [10]:
df

,name,category,description,ingredients,price,rating,image_path
0,Cappuccino,Coffee,A rich and creamy cappuccino made with freshly...,"[Espresso, Steamed Milk, Milk Foam]",4.50,4.7,cappuccino.jpg
1,Jumbo Savory Scone,Bakery,"Deliciously flaky and buttery, this jumbo savo...","[Flour, Butter, Cheese, Herbs, Baking Powder, ...",3.25,4.3,SavoryScone.webp
2,Latte,Coffee,"Smooth and creamy, our latte combines rich esp...","[Espresso, Steamed Milk, Milk Foam]",4.75,4.8,Latte.jpg
3,Chocolate Chip Biscotti,Bakery,"Crunchy and delightful, this chocolate chip bi...","[Flour, Sugar, Chocolate Chips, Eggs, Almonds,...",2.50,4.6,chocolat_biscotti.jpg
4,Espresso shot,Coffee,"A bold shot of rich espresso, our espresso is ...",[Espresso],2.00,4.9,Espresso_shot.webp
5,Hazelnut Biscotti,Bakery,These delicious hazelnut biscotti are perfect ...,"[Flour, Sugar, Hazelnuts, Eggs, Baking Powder]",2.75,4.4,Hazelnut_Biscotti.jpg
6,Chocolate Croissant,Bakery,"Flaky and buttery, our chocolate croissant is ...","[Flour, Butter, Chocolate, Yeast, Sugar, Salt]",3.75,4.8,Chocolate_Croissant.jpg
7,Dark chocolate,Drinking Chocolate,"Rich and indulgent, our dark chocolate drinkin...","[Cocoa Powder, Sugar, Milk]",5.00,4.7,Dark_chocolate.jpg
8,Cranberry Scone,Bakery,This delightful cranberry scone combines sweet...,"[Flour, Butter, Cranberries, Sugar, Baking Pow...",3.50,4.5,Cranberry_Scone.jpg
9,Croissant,Bakery,"Our classic croissant is flaky and buttery, of...","[Flour, Butter, Yeast, Sugar, Salt]",3.25,4.7,Croissant.jpg


In [11]:
df['text'] =  df['name']+" : "+df['description'] + \
                " -- Ingredients: " + df['ingredients'].astype(str) + \
                " -- Price: " + df['price'].astype(str) + \
                " -- rating: " + df['rating'].astype(str) 

In [12]:
df['text'].head(5)

0    Cappuccino : A rich and creamy cappuccino made...
1    Jumbo Savory Scone : Deliciously flaky and but...
2    Latte : Smooth and creamy, our latte combines ...
3    Chocolate Chip Biscotti : Crunchy and delightf...
4    Espresso shot : A bold shot of rich espresso, ...
Name: text, dtype: object

In [13]:
texts=df['text'].tolist()

In [14]:
texts[0]


"Cappuccino : A rich and creamy cappuccino made with freshly brewed espresso, steamed milk, and a frothy milk cap. This delightful drink offers a perfect balance of bold coffee flavor and smooth milk, making it an ideal companion for relaxing mornings or lively conversations. -- Ingredients: ['Espresso', 'Steamed Milk', 'Milk Foam'] -- Price: 4.5 -- rating: 4.7"

In [15]:
with open("../products/Merry's_way_about_us.txt") as f:
    merry_way=f.read()

merry="Coffe shop about: "+merry_way

In [16]:
with open("../products/menu_items_text.txt") as f:
    menu=f.read()

menu="menu: "+menu

In [17]:
texts.append(merry)

In [18]:
texts.remove(menu)


ValueError: list.remove(x): x not in list

In [21]:
texts

["Cappuccino : A rich and creamy cappuccino made with freshly brewed espresso, steamed milk, and a frothy milk cap. This delightful drink offers a perfect balance of bold coffee flavor and smooth milk, making it an ideal companion for relaxing mornings or lively conversations. -- Ingredients: ['Espresso', 'Steamed Milk', 'Milk Foam'] -- Price: 4.5 -- rating: 4.7",
 "Jumbo Savory Scone : Deliciously flaky and buttery, this jumbo savory scone is filled with herbs and cheese, creating a mouthwatering experience. Perfect for a hearty snack or a light lunch, it pairs beautifully with your favorite coffee or tea. -- Ingredients: ['Flour', 'Butter', 'Cheese', 'Herbs', 'Baking Powder', 'Salt'] -- Price: 3.25 -- rating: 4.3",
 "Latte : Smooth and creamy, our latte combines rich espresso with velvety steamed milk, creating a perfect balance of flavor and texture. Enjoy it as a comforting treat any time of day, whether you're starting your morning or taking a midday break. -- Ingredients: ['Espre

In [22]:
output_embeddings=[get_embedding(model_name,model_kwargs,encode_kwargs,x) for x in texts]

In [23]:
index_name = "coffeeshop"

pc.create_index(
    name=index_name,
    dimension=384, # Replace with your model dimensions
    metric="cosine", # Replace with your model metric
    spec=ServerlessSpec(
        cloud="aws",
        region="us-east-1"
    ) 
)

In [24]:
from time import time
# Wait for the index to be ready
while not pc.describe_index(index_name).status['ready']:
    time.sleep(1)

index=pc.Index(index_name)

vectors=[]
""" 
id-capachino,values-embedding,metadata-text

"""

for text,e in zip(texts,output_embeddings):
    entry=text.split(":")[0]
    vectors.append(
        {
        "id":entry,
        "values":e,
        "metadata":{'text':text}}
    )





index.upsert(
    vectors=vectors,
    namespace="ns1"
)


{'upserted_count': 19}

# Get Closest documents

In [28]:
client=get_embedding(model_name,model_kwargs,encode_kwargs,"what is the shop name?")

In [30]:
results=index.query(
    namespace="ns1",
    vector=client,
    top_k=3,
    include_metadata=True,
    include_values=False
    
    
)

In [31]:
results

{'matches': [{'id': 'Coffe shop about',
              'metadata': {'text': 'Coffe shop about: Welcome to Perera and '
                                   'daughters Coffee, your neighborhood coffee '
                                   'shop located in colombo, SriLanka. At '
                                   "Templer's road, we believe that coffee is "
                                   'more than just a drinkâ€”itâ€™s an '
                                   'experience, a moment of joy, and a way to '
                                   'connect with others.\n'
                                   '\n'
                                   'Our Story\n'
                                   'Founded in 2020, Perera and daughters '
                                   'started as a small family-owned cafÃ© with '
                                   'one mission: to share the love of quality, '
                                   'ethically-sourced coffee with our '
                                 